In [1]:
import pandas as pd
import math
import numpy as np
import matplotlib.pyplot as plt
import statsmodels.api as sm
from scipy.stats import norm
from statsmodels.sandbox.regression.gmm import GMM
from statsmodels.base.model import GenericLikelihoodModel

In [2]:
data = pd.read_csv('data.csv')

#pre-processing to calculate outside good shares
shares = data[['Market_ID','Inside Good Share','PPO']].copy()
shares['PPO Share'] = data['Inside Good Share'] * data['PPO']
shares['HMO Share'] = data['Inside Good Share'] * (1 - data['PPO'])

group_shares = shares.groupby('Market_ID').sum()
group_shares['Outside Good Share'] = 1 - group_shares['Inside Good Share']

data = pd.merge(data,group_shares[['Outside Good Share','PPO Share','HMO Share']], 
                right_index=True, left_on = 'Market_ID')

data['Nest Market Size'] = ( (data['Inside Good Share'] * data['PPO']) / data['PPO Share'] 
                            + (data['Inside Good Share'] * (1 - data['PPO'])) /data['HMO Share'])

#calculate ln(Inside Good Share)
data['ln(Inside Good Share)'] = np.log( data['Inside Good Share']/data['Nest Market Size'] ) 
data = data.fillna(0) #necessary for log(1), unsure why

print data.min()

Market_ID                1.000000
Plan_ID                  1.000000
PPO                      0.000000
Network Score            0.780000
Satisfaction Score       0.725000
Premium                  2.269728
Inside Good Share        0.037670
Outside Good Share       0.096070
PPO Share                0.000000
HMO Share                0.000000
Nest Market Size         0.000000
ln(Inside Good Share)   -2.428602
dtype: float64


In [3]:
#first estimate using logit
class logit(GMM):
    
    def __init__(self, *args, **kwds):
        # set appropriate counts for moment conditions and parameters
        super(logit, self).__init__(*args, **kwds)

        
    def momcond(self, params):
        #unwrap stuff
        shares = np.array(self.endog).transpose()
        exog = np.array(self.exog)
        instr = np.array(self.instrument)
        
        lshare = np.log(shares[0]) -  np.log(shares[1])
        lshare = lshare.transpose()
       
        lshare_fit = np.matmul(exog,params) #linear equation    
        
        xi = lshare_fit - lshare
        g = instr * xi[:, np.newaxis]
        
        return g

In [4]:
#calculate hausmann insturments
mkt_dum = pd.get_dummies(data['Market_ID'],prefix='mkt',drop_first=True)
plan_dum = pd.get_dummies(data['Plan_ID'],prefix='plan',drop_first=True)
hausman_instr = plan_dum

#set up data for logit
y = data[['Inside Good Share','Outside Good Share']]

# add ln(inside good share) as regressor like formula
x_nested = data[['Network Score','Satisfaction Score','PPO','Premium','ln(Inside Good Share)']]

In [5]:
#set up and run model
beta_nested = np.full(len(x_nested.columns),1)
model = logit(y , x_nested, hausman_instr)
result = model.fit(beta_nested, maxiter=2, optim_method='nm', wargs=dict(centered=False))

print(result.summary())

Optimization terminated successfully.
         Current function value: 0.000008
         Iterations: 524
         Function evaluations: 850
Optimization terminated successfully.
         Current function value: 0.004920
         Iterations: 153
         Function evaluations: 267
                                             logit Results                                             
Dep. Variable:     ['Inside Good Share', 'Outside Good Share']   Hansen J:                        16.24
Model:                                                   logit   Prob (Hansen J):                0.0931
Method:                                                    GMM                                         
Date:                                         Tue, 09 Oct 2018                                         
Time:                                                 17:09:27                                         
No. Observations:                                         3300                                  

In [6]:
#compute market shares

#compute exp(delta_j)
delta_nest = data.copy()[['Market_ID','Plan_ID','PPO','Premium']]
x = np.array(x_nested[['Network Score','Satisfaction Score','PPO','Premium']])
beta = result.params[:-1]
alpha = abs(beta[3])
sigma = abs(result.params[-1]) #not sure about this, feel like it should be positive?
delta_nest['exp_delta'] = np.exp(np.matmul(x,beta)/(1-sigma))

#compute Dg = sum_j|g exp(delta_j)
s1 = delta_nest[['Market_ID','exp_delta','PPO']].copy()
s1['PPO_delta'] = delta_nest['exp_delta'] * delta_nest['PPO']
s1['HMO_delta'] = delta_nest['exp_delta'] * (1 - delta_nest['PPO'])

group_s1 = s1.groupby('Market_ID').sum()
group_s1['PP0_delta_sigma'] = group_s1['PPO_delta']**(sigma)
group_s1['HMO_delta_sigma'] = group_s1['HMO_delta']**(sigma)
group_s1['sum_g'] = group_s1['PPO_delta']**(1-sigma) + group_s1['HMO_delta']**(1-sigma)
#not sure if there should be a 1 at the end of this... (for outside good?)

delta_nest = pd.merge(delta_nest,group_s1[['PPO_delta','HMO_delta','PP0_delta_sigma','sum_g']], 
                right_index=True, left_on = 'Market_ID')

#compute sum_g Dg^(1-sigma)
delta_nest['denom'] = ( (1 - delta_nest['PPO'])*delta_nest['HMO_delta']  +
                 delta_nest['PPO']*delta_nest['PPO_delta']) * (delta_nest['sum_g'])

delta_nest['fitted_share'] = delta_nest['exp_delta']/delta_nest['denom']

In [7]:
print delta_nest['fitted_share'].max(),delta_nest['fitted_share'].min(),delta_nest['fitted_share'].mean(), np.sqrt(delta_nest['fitted_share'].var())

0.5186716445245331 0.01170248845783048 0.0732291327029 0.07212266746307026


In [8]:
#compute total share of each nest
fitted_shares = delta_nest[['Market_ID','fitted_share','PPO']].copy()
fitted_shares['fitted_ppo'] = delta_nest['fitted_share'] * delta_nest['PPO']
fitted_shares['fitted_hmo'] = delta_nest['fitted_share'] * (1 - delta_nest['PPO'])

group_s2 = fitted_shares.groupby('Market_ID').sum()
delta_nest = pd.merge(delta_nest,group_s2[['fitted_ppo','fitted_hmo']], 
                right_index=True, left_on = 'Market_ID')

delta_nest['fitted_nest_share'] = ( (delta_nest['fitted_share'] * delta_nest['PPO']) / delta_nest['fitted_ppo'] 
                            + (delta_nest['fitted_share'] * (1 - delta_nest['PPO'])) /delta_nest['fitted_hmo'])

In [9]:
#aggregate elasticities
delta_nest = delta_nest.groupby('Plan_ID').mean()

#diagonal formula
nest_shares = np.array(delta_nest['fitted_nest_share'])
sharesn = np.array(delta_nest['fitted_share'])
prices = np.array(delta_nest['Premium'])
ppo = np.array([delta_nest['PPO']]) #this one is a matrix

#selector matrices
inside_nest = np.matmul(ppo.transpose(),ppo) + np.matmul((1-ppo.transpose()),(1-ppo))
outside_nest = 1 - inside_nest 
own_price = np.identity(len(sharesn))

#elasticity variables
inside_elasticity = np.matmul( np.array( [ (sigma/(1-sigma) * nest_shares + sharesn)/sharesn ] ).transpose(),
          np.array([alpha*prices*sharesn]))

own_elasticity = -alpha*((1/(1-sigma)) - sigma/(1-sigma) * nest_shares - sharesn)*prices

outside_elasticity =  sharesn * alpha * prices

nest_elasticity = own_price*own_elasticity + inside_nest*inside_elasticity + outside_nest*outside_elasticity


print outside_nest
print nest_elasticity[1][1]
print nest_elasticity[0][1],nest_elasticity[2][1] #should be the same (outside nest)
print nest_elasticity[4][1],nest_elasticity[5][1] #should be different

[[0. 1. 0. 0. 1. 1. 0. 1. 0. 1. 1. 0. 0. 1. 0. 1.]
 [1. 0. 1. 1. 0. 0. 1. 0. 1. 0. 0. 1. 1. 0. 1. 0.]
 [0. 1. 0. 0. 1. 1. 0. 1. 0. 1. 1. 0. 0. 1. 0. 1.]
 [0. 1. 0. 0. 1. 1. 0. 1. 0. 1. 1. 0. 0. 1. 0. 1.]
 [1. 0. 1. 1. 0. 0. 1. 0. 1. 0. 0. 1. 1. 0. 1. 0.]
 [1. 0. 1. 1. 0. 0. 1. 0. 1. 0. 0. 1. 1. 0. 1. 0.]
 [0. 1. 0. 0. 1. 1. 0. 1. 0. 1. 1. 0. 0. 1. 0. 1.]
 [1. 0. 1. 1. 0. 0. 1. 0. 1. 0. 0. 1. 1. 0. 1. 0.]
 [0. 1. 0. 0. 1. 1. 0. 1. 0. 1. 1. 0. 0. 1. 0. 1.]
 [1. 0. 1. 1. 0. 0. 1. 0. 1. 0. 0. 1. 1. 0. 1. 0.]
 [1. 0. 1. 1. 0. 0. 1. 0. 1. 0. 0. 1. 1. 0. 1. 0.]
 [0. 1. 0. 0. 1. 1. 0. 1. 0. 1. 1. 0. 0. 1. 0. 1.]
 [0. 1. 0. 0. 1. 1. 0. 1. 0. 1. 1. 0. 0. 1. 0. 1.]
 [1. 0. 1. 1. 0. 0. 1. 0. 1. 0. 0. 1. 1. 0. 1. 0.]
 [0. 1. 0. 0. 1. 1. 0. 1. 0. 1. 1. 0. 0. 1. 0. 1.]
 [1. 0. 1. 1. 0. 0. 1. 0. 1. 0. 0. 1. 1. 0. 1. 0.]]
-4.381805375785701
0.33923356160990165 0.33923356160990165
0.4327571052033485 0.4321921035146931
